# Transmission lines

PowerGenome compiles information on the transmission constraints between model regions and the centroid-to-centroid distance between model regions.

In [6]:
%load_ext autoreload
%autoreload 2

In [7]:
from pathlib import Path

import pandas as pd
from powergenome.generators import load_ipm_shapefile
from powergenome.GenX import (
    network_line_loss,
    network_max_reinforcement,
    network_reinforcement_cost,
    add_cap_res_network
)
from powergenome.transmission import (
    agg_transmission_constraints,
    transmission_line_distance,
)
from powergenome.util import init_pudl_connection, load_settings

## Import settings

In [14]:
pudl_engine, pudl_out, pg_engine = init_pudl_connection()
cwd = Path.cwd()

settings_path = (
    cwd.parent / "example_system" / "test_settings.yml"
)
settings = load_settings(settings_path)
settings["input_folder"] = settings_path.parent / settings["input_folder"]

## Aggregate transmission constraints
EPA provides bulk transmission constraints between IPM regions. PowerGenome combines these constraints across aggregated model regions. The required settings parameters are:
- `model_regions`
- `region_aggregations`

In [4]:
settings["model_regions"]

['CA_N', 'CA_S', 'WECC_AZ']

In [5]:
settings["region_aggregations"]

{'CA_N': ['WEC_CALN', 'WEC_BANC'],
 'CA_S': ['WEC_LADW', 'WECC_SCE', 'WEC_SDGE', 'WECC_IID']}

In [47]:
transmission = agg_transmission_constraints(pg_engine=pg_engine, settings=settings)

In [23]:
transmission

,Network_Lines,z1,z2,z3,Line_Max_Flow_MW,Line_Min_Flow_MW,Transmission Path Name
0,1,1,-1,0,3675.0,-3000.0,CA_N_to_CA_S
1,2,0,1,-1,3767.0,-4794.0,CA_S_to_WECC_AZ


## Add centroid-to-centroid distance

In [7]:
model_regions_gdf = load_ipm_shapefile(settings)
model_regions_gdf

/opt/miniconda3/envs/pg_pudl_update/lib/python3.9/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/opt/miniconda3/envs/pg_pudl_update/lib/python3.9/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


,IPM_Region,geometry,model_region
0,WEC_BANC,"MULTIPOLYGON (((-137062.964 -20577.766, -13352...",CA_N
1,WEC_CALN,"MULTIPOLYGON (((-26905.041 -343681.519, -26858...",CA_N
2,WEC_LADW,"MULTIPOLYGON (((147531.138 -578110.363, 147523...",CA_S
3,WEC_SDGE,"MULTIPOLYGON (((261508.595 -578272.714, 261412...",CA_S
4,WECC_AZ,"POLYGON ((971403.331 -56900.550, 1043399.918 -...",WECC_AZ
5,WECC_IID,"POLYGON ((499663.896 -494815.927, 497706.026 -...",CA_S
6,WECC_SCE,"MULTIPOLYGON (((43384.114 -525298.005, 50284.5...",CA_S


In [8]:
transmission_line_distance(
    trans_constraints_df=transmission,
    ipm_shapefile=model_regions_gdf,
    settings=settings,
)

,Network_Lines,z1,z2,z3,Line_Max_Flow_MW,Line_Min_Flow_MW,Transmission Path Name,distance_mile
0,1,1,-1,0,3675.0,-3000.0,CA_N_to_CA_S,336.613945
1,2,0,1,-1,3767.0,-4794.0,CA_S_to_WECC_AZ,320.961288


## Line loss, reinforcement costs, max reinforcement
These functions use the settings parameters:
- `transmission_investment_costs.tx.capex_mw_mile.<model_region>` (a nested dictionary with a single value for each region. Transmission between regions uses an average of the two values)
- `transmission_investment_costs.tx.wacc` (a numeric value for the weighted average cost of capital within the nested dictionary)
- `transmission_investment_costs.tx.investment_years` (a numeric value for the investment timeframe within the nested dictionary)
- `tx_line_loss_100_miles`
- `tx_expansion_per_period`

Transmission costs within a model region are difficult to automate, because it is impossible to map a user-defined region to something like ReEDS cost regions. But I have created [a mapping of IPM regions to ReEDS regions](https://github.com/gschivley/pg_misc/blob/master/create_clusters/site_interconnection_costs.py#L32-L155) and the associated transmission costs.

In [10]:
settings["transmission_investment_cost"]["tx"]

{'capex_mw_mile': {'CA_N': 3037.5, 'CA_S': 3037.5, 'WECC_AZ': 1350},
 'wacc': 0.069,
 'investment_years': 60}

In [11]:
settings["tx_line_loss_100_miles"]

0.01

In [12]:
settings["tx_expansion_per_period"]

1.0

In [8]:
network_line_loss(transmission=transmission, settings=settings)

,Network_lines,z1,z2,z3,Line_Max_Flow_MW,Line_Min_Flow_MW,Transmission Path Name,distance_mile,Line_Loss_Percentage
0,1,1,-1,0,10735.0,-9858.0,CA_N_to_CA_S,556.492226,0.055649
1,2,1,0,-1,250.0,-250.0,CA_N_to_WECC_AZ,629.980950,0.062998
2,3,0,1,-1,10952.0,-11921.0,CA_S_to_WECC_AZ,203.571643,0.020357


In [9]:
network_reinforcement_cost(transmission=transmission, settings=settings)

,Network_lines,z1,z2,z3,Line_Max_Flow_MW,Line_Min_Flow_MW,Transmission Path Name,distance_mile,Line_Loss_Percentage,Line_Reinforcement_Cost_per_MW_yr
0,1,1,-1,0,10735.0,-9858.0,CA_N_to_CA_S,556.492226,0.055649,122706.0
1,2,1,0,-1,250.0,-250.0,CA_N_to_WECC_AZ,629.980950,0.062998,100324.0
2,3,0,1,-1,10952.0,-11921.0,CA_S_to_WECC_AZ,203.571643,0.020357,32419.0


In [10]:
network_max_reinforcement(transmission=transmission, settings=settings)

,Network_lines,z1,z2,z3,Line_Max_Flow_MW,Line_Min_Flow_MW,Transmission Path Name,distance_mile,Line_Loss_Percentage,Line_Reinforcement_Cost_per_MW_yr,Line_Max_Reinforcement_MW
0,1,1,-1,0,10735.0,-9858.0,CA_N_to_CA_S,556.492226,0.055649,122706.0,10735.0
1,2,1,0,-1,250.0,-250.0,CA_N_to_WECC_AZ,629.980950,0.062998,100324.0,250.0
2,3,0,1,-1,10952.0,-11921.0,CA_S_to_WECC_AZ,203.571643,0.020357,32419.0,10952.0


In [90]:
transmission = agg_transmission_constraints(pg_engine=pg_engine, settings=settings)

In [91]:
add_cap_res_network(transmission, settings)

,Network_Lines,z1,z2,z3,Line_Max_Flow_MW,Line_Min_Flow_MW,Transmission Path Name,CapRes_1,CapRes_2,DerateCapRes_1,DerateCapRes_2,CapRes_Excl_1,CapRes_Excl_2
0,1,1,-1,0,3675.0,-3000.0,CA_N_to_CA_S,0,0,0.95,0.95,0,0
1,2,0,1,-1,3767.0,-4794.0,CA_S_to_WECC_AZ,1,1,0.95,0.95,1,-1


In [57]:
def label_cap_res_lines(path_names, dest_regions: list) -> list:
    cap_res_list = []
    for name in path_names:
        s_r = name.split("_to_")[0]
        e_r = name.split("_to_")[-1]
        if (s_r in dest_regions or e_r in dest_regions) and not (
            s_r in dest_regions and e_r in dest_regions
        ):
            cap_res_list.append(1)
        else:
            cap_res_list.append(0)

    return cap_res_list

In [89]:
tx_df

,Network_Lines,z1,z2,z3,Line_Max_Flow_MW,Line_Min_Flow_MW,Transmission Path Name,CapRes_Excl_1,CapRes_Excl_2
0,1,1,-1,0,3675.0,-3000.0,CA_N_to_CA_S,0,0
1,2,0,1,-1,3767.0,-4794.0,CA_S_to_WECC_AZ,1,-1
